In [1]:
import sys
import pandas as pd
# sys.path is a list of absolute path strings
sys.path.append('C:\Projects\Private\PropStar')
from normalization.normalize import Normalize


In [5]:
dataset_group = pd.read_csv('../../datasets/indoor_user_movement_rss_data/groups/MovementAAL_DatasetGroup.csv')
dataset_group['sequence_ID'] = dataset_group['#sequence_ID']
dataset_group.drop('#sequence_ID', axis=1, inplace=True)

In [8]:
dataset_group

,dataset_ID,sequence_ID
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
...,...,...
309,3,310
310,3,311
311,3,312
312,3,313


In [12]:
paths = pd.read_csv('../../datasets/indoor_user_movement_rss_data/groups/MovementAAL_Paths.csv')
paths['sequence_ID'] = paths['#sequence_ID']
paths.drop('#sequence_ID', axis=1, inplace=True)

In [13]:
paths

,path_ID,sequence_ID
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
...,...,...
309,6,310
310,6,311
311,6,312
312,6,313


In [15]:
target = pd.read_csv('../../datasets/indoor_user_movement_rss_data/MovementAAL_target.csv')
target['class_label'] = target[' class_label']
target['sequence_ID'] = target['#sequence_ID']
target.drop(['#sequence_ID', ' class_label'], axis=1, inplace=True)

In [16]:
target.columns

Index(['class_label', 'sequence_ID'], dtype='object')

In [17]:
target

,class_label,sequence_ID
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
...,...,...
309,-1,310
310,-1,311
311,-1,312
312,-1,313


In [19]:
import os 

files = os.listdir('../../datasets/indoor_user_movement_rss_data/dataset')
movements_df = pd.DataFrame()

In [21]:
for file in files:
    if file.startswith('.'):
        continue
    seq_id = file.split("_")[2].replace(".csv", "")
    file_df = pd.read_csv(f"../../datasets/indoor_user_movement_rss_data/dataset/{file}")
    file_df['seq_id'] = seq_id
    movements_df = pd.concat([movements_df, file_df])

In [22]:
movements_df.reset_index(inplace=True)
movements_df.drop("index", axis=1, inplace=True)

In [23]:
movements_df.reset_index(inplace=True, names='id')

In [24]:
movements_df['RSS_anchor1'] = movements_df['#RSS_anchor1']
movements_df.drop('#RSS_anchor1', axis=1, inplace=True)

In [25]:
movements_df['seq_id'] = pd.to_numeric(movements_df['seq_id'])

In [26]:
len(movements_df['seq_id'].unique())

314

In [27]:
movements_df

,id,RSS_anchor2,RSS_anchor3,RSS_anchor4,seq_id,RSS_anchor1
0,0,-0.48,0.28571,0.30,1,-0.90476
1,1,-0.32,0.14286,0.30,1,-0.57143
2,2,-0.28,-0.14286,0.35,1,-0.38095
3,3,-0.20,-0.47619,0.35,1,-0.28571
4,4,-0.20,0.14286,-0.20,1,-0.14286
...,...,...,...,...,...,...
13192,13192,0.28,-0.76190,-0.55,99,0.19048
13193,13193,0.40,-1.00000,-0.75,99,0.19048
13194,13194,0.24,-1.00000,-0.65,99,0.28571
13195,13195,0.20,-0.85714,-0.55,99,0.33333


In [28]:
norm = Normalize(denormalized_table=target, denormalized_table_name="target")

In [29]:
norm.entity_set['target'].drop("id", axis=1, inplace=True)

In [30]:
norm.entity_set['movements'] = movements_df

In [31]:
norm.entity_set['dataset_groups'] = dataset_group
norm.entity_set['paths'] = paths

In [32]:
norm.primary_keys['target'] = "sequence_ID"
norm.primary_keys['movements'] = "id"
norm.primary_keys['dataset_groups'] = 'sequence_ID'
norm.primary_keys['paths'] = 'sequence_ID'

In [33]:
norm.primary_keys

{'target': 'sequence_ID',
 'movements': 'id',
 'dataset_groups': 'sequence_ID',
 'paths': 'sequence_ID'}

In [34]:
norm.add_custom_relation(parent_table_name='target', parent_table_column='sequence_ID', child_table_name='movements', child_table_column='seq_id')
norm.add_custom_relation(parent_table_name='target', parent_table_column='sequence_ID', child_table_name='dataset_groups', child_table_column='sequence_ID')
norm.add_custom_relation(parent_table_name='target', parent_table_column='sequence_ID', child_table_name='paths', child_table_column='sequence_ID')

In [35]:
norm.relations

{'target': {'movements': ('sequence_ID', 'seq_id'),
  'dataset_groups': ('sequence_ID', 'sequence_ID'),
  'paths': ('sequence_ID', 'sequence_ID')}}

In [36]:
norm.persist_entity_set(database_name='indoor_user_movement')

2024-07-03 19:24:28,809 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)


[07-03 19:24:28] p38176 {C:\Projects\Private\PropStar\venv\lib\site-packages\sqlalchemy\engine\base.py:1897} INFO - SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)


2024-07-03 19:24:28,811 INFO sqlalchemy.engine.Engine [raw sql] ()


[07-03 19:24:28] p38176 {C:\Projects\Private\PropStar\venv\lib\site-packages\sqlalchemy\engine\base.py:1902} INFO - [raw sql] ()


2024-07-03 19:24:28,814 INFO sqlalchemy.engine.Engine SELECT schema_name()


...

[07-03 19:24:30] p38176 {C:\Projects\Private\PropStar\venv\lib\site-packages\sqlalchemy\engine\base.py:1897} INFO - ALTER TABLE indoor_user_movement.paths ADD CONSTRAINT FK_paths_target FOREIGN KEY (sequence_ID) REFERENCES indoor_user_movement.target(sequence_ID);


2024-07-03 19:24:30,221 INFO sqlalchemy.engine.Engine [generated in 0.00178s] ()


[07-03 19:24:30] p38176 {C:\Projects\Private\PropStar\venv\lib\site-packages\sqlalchemy\engine\base.py:1902} INFO - [generated in 0.00178s] ()


2024-07-03 19:24:30,228 INFO sqlalchemy.engine.Engine ROLLBACK


[07-03 19:24:30] p38176 {C:\Projects\Private\PropStar\venv\lib\site-packages\sqlalchemy\engine\base.py:1113} INFO - ROLLBACK


In [26]:
norm.entity_set['movements'].dtypes

id                int64
 RSS_anchor2    float64
 RSS_anchor3    float64
 RSS_anchor4    float64
seq_id            int64
RSS_anchor1     float64
dtype: object

In [27]:
target

,class_label,sequence_ID
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
...,...,...
309,-1,310
310,-1,311
311,-1,312
312,-1,313
